In [6]:
import numpy as np
from numpy.linalg import norm
import tensorflow as tf
import pandas as pd
from functions import plot_fids


In [18]:
arr = np.array([[-13.8856291967975,-66.3281814953512,247.617811143251],
[20.9187637544014,-23.0027976452465,257.309409366197],
[60.3827961786212,-17.7267229935585,257.281115278552],
[65.5131942085598,-56.0851307744565,247.743207934783],
[22.1143071892556,-30.4731006408006,203.835899831461],
[57.8630109021893,5.24918895656779,160.127783361582],
[-20.7315857862206,-9.59489945323305,179.500194806362],
[-16.0539816151494,-43.3458676545516,149.793031304348]]).astype('float16')
fig = plot_fids(arr)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [2]:
df = pd.read_csv('788598_collect.csv')
tool_pos = df[['toolx','tooly','toolz']].to_numpy()
ref_pos = df[['refx','refy','refz']].to_numpy()

In [3]:
a = tool_pos.transpose()
b = ref_pos

# Run on the GPU
c = a@b
c

array([[ 1.09490801e+06,  1.81459279e+06, -2.61863367e+07],
       [ 8.43311889e+06,  1.39761882e+07, -2.01689989e+08],
       [-1.20492067e+08, -1.99691222e+08,  2.88173890e+09]])

In [4]:
def unit(vector):
    """ Returns the unit vector of the vector.  """
    if np.linalg.norm(vector)== 0:
        return np.array([0,0,0])
    else:
        return vector / np.linalg.norm(vector)

def distanceFromLine(p1,p2,p3):
    # distance of a point p3 from the line formed using points p1 and p2
    if norm(p2-p1) == 0:
        return 100
    else:
        d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)
        return d


In [5]:
def compute_tre(vector,target):
    

    

    # add noise to system
    # vector = vector + noise

    
    centroid = np.mean(vector,axis=0)
    

    axis =[]
    temp_vec =  unit(vector[1] -  centroid)

    temp_vec2 = tnp.cross(temp_vec,unit(vector[2]-centroid))
    temp_vec3 = tnp.cross(temp_vec,temp_vec2)
    axis.append(temp_vec)
    axis.append(temp_vec2)
    axis.append(temp_vec3)


    # fiducials to principal axis
    # target from principal axis 
    tp = []
    tp.append(distanceFromLine(centroid,centroid + axis[0],target))
    tp.append(distanceFromLine(centroid,centroid + axis[1],target))
    tp.append(distanceFromLine(centroid,centroid + axis[2],target))

    fp = []
    f = np.zeros(3)
    for i in range(3):
        sum = 0
        for j in range(len(vector)):
            fp = np.around(distanceFromLine(centroid,centroid+axis[i],vector[j]),3)
            # print(fp)
            sum = sum + fp**2
        if np.round(sum/len(vector)) == 0:
            f[i]=10000
            # print(f[i])
        else:
            f[i] = tp[i]**2/(sum/len(vector))
            
        
        

    fle = 0.3
    tre = (fle**2/len(vector))*(np.mean(f)+1)
    return np.sqrt(np.around(tre,3))

In [10]:
p1 = np.array([1,3,4]).astype('float16')
p2 = np.array([3,4,5]).astype('float16')

p3 = np.array([4,7,8]).astype('float16')

fiducial = [p1,p2,p3]
tre =compute_tre (fiducial,[0,0,0])
print(tre)

0.5300943312279429


In [7]:
def unitTens(vector):
    return vector / tf.raw_ops.EuclideanNorm(input=vector,axis=0)


def DistanceFromLineTens(p1,p2,p3):
    p = p2-p1
    if tf.raw_ops.EuclideanNorm(input=p,axis=0) == 0:
        return 100
    else:
        d = tf.raw_ops.EuclideanNorm(input=tf.linalg.cross(p2-p1,p1-p3),axis=0)/tf.raw_ops.EuclideanNorm(input=p,axis=0)
        return d
    
def compute_treT(vector,target):
    centroid = tnp.mean(vector,axis = 0).astype('float16')
    axis =[]
    temp_vec =  unitTens(vector[1] -  centroid)

    temp_vec2 = tnp.cross(temp_vec,unit(vector[2]-centroid))
    temp_vec3 = tnp.cross(temp_vec,temp_vec2)
    axis.append(temp_vec)
    axis.append(temp_vec2)
    axis.append(temp_vec3)


    # fiducials to principal axis
    # target from principal axis 
    tp = []
    tp.append(DistanceFromLineTens(centroid,centroid + axis[0],target))
    tp.append(DistanceFromLineTens(centroid,centroid + axis[1],target))
    tp.append(DistanceFromLineTens(centroid,centroid + axis[2],target))

    fp = []
    f = np.zeros(3)
    for i in range(3):
        sum = 0
        for j in range(len(vector)):
            fp = tnp.around(DistanceFromLineTens(centroid,centroid+axis[i],vector[j]),3)
            # print(fp)
            sum = sum + fp**2
        if tnp.round(sum/len(vector)) == 0:
            f[i]=10000
            # print(f[i])
        else:
            f[i] = tp[i]**2/(sum/len(vector))
            
        
        

    fle = 0.3
    tre = (fle**2/len(vector))*(tnp.mean(f)+1).astype('float16')
    return tf.math.sqrt(tnp.around(tre,3))


In [9]:
gpus = tf.config.list_physical_devices('GPU')
with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
    p1 = tf.constant([1,3,4],dtype='float16')
    p2 = tf.constant([3,4,5],dtype='float16')
    p3 = tf.constant([4,7,8],dtype='float16')


    fiducial = [p1,p2,p3]
    tre =compute_treT(fiducial,[0,0,0])
    print(tre)

tf.Tensor(0.5303, shape=(), dtype=float16)
